In [1]:
# from zipfile import ZipFile 


# # specifying the zip file name 
# file_name = "C:\proj\\trafficnet_dataset_v1.zip"


  
# #opening the zip file in READ mode
# with ZipFile(file_name, 'r') as zip: 
#     # printing all the contents of the zip file 
#     zip.printdir() 
  
#     # extracting all the files 
#     print('Extracting all the files now...') 
#     zip.extractall() 
#     print('Done!') 
    

In [42]:
!pip install scipy
!pip install matplotlib

In [43]:
#Import all necessary libraries
from PIL import Image
from tensorflow import keras
from tensorflow.keras.layers import Dense,Flatten,GlobalAveragePooling2D,Input,Dropout,Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.callbacks import EarlyStopping,ReduceLROnPlateau 
import scipy

In [45]:
#Augment the images
datagen = ImageDataGenerator(
  featurewise_center=False,  # set input mean to 0 over the dataset
  samplewise_center=False,  # set each sample mean to 0
  featurewise_std_normalization=False,  # divide inputs by std of the dataset
  samplewise_std_normalization=False,  # divide each input by its std
  zca_whitening=True,  # apply ZCA whitening
  rotation_range=90,  # randomly rotate images in the range (degrees, 0 to 180)
  width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
  height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
  horizontal_flip=True,  # randomly flip images
  vertical_flip=False)  # randomly flip images
  

In [46]:
#Train dataset
trainDatagen = datagen.flow_from_directory(directory='C:/proj/trafficnet_dataset_v1/train/',
                                           target_size=(224, 224),
                                           class_mode = 'sparse',
                                           batch_size = 64)

Found 3600 images belonging to 4 classes.


In [47]:
#Validation dataset
valDatagen = datagen.flow_from_directory(directory='C:/proj/trafficnet_dataset_v1/test/',
                                           target_size=(224, 224),
                                           class_mode = 'sparse',
                                           batch_size = 64)

Found 800 images belonging to 4 classes.


In [48]:
#Assign numbers to each category
trainDatagen.class_indices

{'accident': 0, 'dense_traffic': 1, 'fire': 2, 'sparse_traffic': 3}

In [49]:
res_model=ResNet50(include_top=False,weights='imagenet',input_shape=(224,224,3)) #Load in ResNet50

In [50]:
#model1
c1=Conv2D(16,(3,3),activation='relu')(res_model.output)
f1=Flatten()(c1)
fc=Dense(4,activation='softmax')(f1)
model=Model(res_model.inputs,fc)
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_3[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

In [20]:
#Model2
avg=GlobalAveragePooling2D()(res_model.output)
fc1=Dense(512,activation='relu')(avg)
fc2=Dense(4,activation='softmax')(fc1)

model=Model(res_model.inputs, fc2)
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

In [51]:
for ix in range(170):
    model.layers[ix].trainable=False #Make all the 170 layers in this model, untrainable

In [52]:
model.summary() #Get a suammary of the entire model

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_3[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

In [63]:
adam=Adam(learning_rate=0.0001) #Our optimizer will be Adam and our learning rate is 0.0001
model.compile(loss='sparse_categorical_crossentropy',optimizer=adam,metrics=['accuracy']) #Compile the model

In [64]:
earlystop=EarlyStopping(patience=10) #Monitor the performance. If it dips, then stop training
learning_rate_reduce=ReduceLROnPlateau(monitor='val_accuracy',min_lr=0.00001) #Change learning rate if not performing good enough
callbacks=[earlystop,learning_rate_reduce]

In [ ]:
#Start training the model with 60 epochs and train it 2 times

history=model.fit(x=trainDatagen,
                            steps_per_epoch=len(trainDatagen),
                            epochs=60,
                            validation_data=valDatagen,
                            validation_steps=len(valDatagen),
                            shuffle=True,
                            callbacks=callbacks
                          )

Epoch 1/60
57/57 [==============================] - 32s 503ms/step - loss: 0.0252 - accuracy: 0.9897 - val_loss: 0.5866 - val_accuracy: 0.8988 - lr: 1.0000e-04
Epoch 2/60
46/57 [=======================>......] - ETA: 4s - loss: 0.0331 - accuracy: 0.9865

In [ ]:
import matplotlib.pyplot as plt

model.save('C:/proj/model.h5')

#plot this model
plt.plot(history.history['accuracy'],color='black', label='Accuracy')
plt.plot(history.history['loss'],color='blue', label='Loss')
plt.plot(history.history['val_accuracy'],color='yellow', label='Validation Accuracy')
plt.plot(history.history['val_loss'],color='red', label='Validation loss')
plt.legend()
plt.show()

In [38]:
classes = ['accident', 'dense_traffic', 'fire', 'sparse_traffic'] #Make a list of the classes

In [39]:
import tensorflow as tf
from tensorflow.keras.utils import load_img, img_to_array  
from tensorflow.keras.models import load_model
import PIL
import numpy as np

def prediction(path, model, classes): #Make a function to predict an image to test this model

    img = load_img(path) #Load image
    img = img.resize((224, 224)) #Resize image
    img = img_to_array(img) #Convert into an array

    img = img.reshape( -1,224, 224,3) #Reshape it

    pred = model.predict(img) #Predict the image

    return classes[int(np.argmax(pred, axis=1))] #Return the classified labels

In [40]:
#Driver code

print(prediction("C:\proj\\trafficnet_dataset_v1\\test\\fire\images.jpg", model, classes))

1/1 [==============================] - 1s 899ms/step
fire
